In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install pascal-voc-writer
!pip install moviepy
!pip install natsort

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt

In [ ]:
!cp /content/drive/MyDrive/Best/best.pt /content/
!cp /content/drive/MyDrive/Football_dataset/Video.mp4 /content/

In [ ]:
import cv2
import os
os.makedirs('/content/Images', exist_ok=True)
vidcap = cv2.VideoCapture('/content/Video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  success,image = vidcap.read()
  cv2.imwrite("/content/Images/frame%d.jpg" % count, image)     # save frame as JPEG file
  if cv2.waitKey(10) == 27:                     # exit if Escape is hit
      break
  count += 1

In [ ]:
import os
os.chdir('/content/Images/')
for filename in os.listdir('/content/Images/'):
  os.rename(filename, filename.replace('frame', ''))

In [ ]:
import os
import PIL
from PIL import Image, ImageDraw,ImageFont
import torch
import pandas as pd
import cv2
from pascal_voc_writer import Writer
from google.colab import files
import shutil
import glob
from pathlib import Path
from sklearn.cluster import KMeans
import cv2 as cv
import numpy as np
from IPython.display import Image, display
from sklearn.decomposition import PCA
from natsort import natsorted




#defining model with best weights and defining confidance of 50
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/best.pt', force_reload=True)
model.conf = 0.55

#creating cropped folder 
os.makedirs('/content/Cropped_Images/' , exist_ok=True)

a = natsorted(glob.glob('/content/Images/*.jpg'))

print(a)

filename_predictions = []


for i in a:filename_predictions.append(Path(i))

print(filename_predictions)
for i in filename_predictions:
  print(i)
  #create a folder for cropped images
  os.makedirs('/content/Cropped_Images/'+i.stem + "/" , exist_ok=True)
  
  #predicting the model
  results = model(PIL.Image.open(i))
  torch.cuda.empty_cache()

  #pandas values of bounding box
  df=results.pandas().xyxy[0]
  df=df.drop(['confidence'],axis=1)
  df=df[['class','xmin','xmax','ymin','ymax','name']]
  #counting no of bounding box
  count_row = df.shape[0]

  #reading image details:
  im = cv2.imread(i)
  height=im.shape[0]
  width=im.shape[1]
  depth=im.shape[2]

  #finding x,y,h,w: of each bounding Box
  for j in range(0,count_row):
    print(j)
    h=df["ymax"].values[j]-df["ymin"].values[j]
    w=df["xmax"].values[j]-df["xmin"].values[j]
    y=df["ymin"].values[j]
    x=df["xmin"].values[j]
    
    #cropping and saving the file as per number on annotaion
    # try:
      
      #crop = im[int(y+((h-30)/2)):int(y+(h/2)+15), int(x+((w-30)/2)):int((x+(w/2)+15))]
    crop = im[int(y+((h-(h*.30))/2)):int(y+(h/2)+((h*.30)/2)), int(x+((w-(w*.30))/2)):int(x+(w/2)+((w*.30)/2))]
    # except:
    #   print('donkey')
    #   crop = im[int(y):int(y+h), int(x):int(x+w)]

    #crop =im[int(((y+h)/2)-20):int(((y+h)/2)+20),int(((x+w)/2)-20):int(((x+w)/2)+20)]
    #print(f"x={x},y={y},w={w},h={h}")
    #print(f"x1={((x+w)/2)-20},y2={((y+h)/2)-20},x2={((x+w)/2)+20},y2={((y+h)/2)+20}")
    cv2.imwrite("/content/Cropped_Images/"+ i.stem + "/" + str(j) + ".jpg",crop)

  # relative path to search all jpg files
  b = natsorted(glob.glob('/content/Cropped_Images/'+i.stem +'/*.jpg'))
  filename_cropped = []
  for n in b: filename_cropped.append(Path(n))
  
  array_1=np.array([])
  final=[]
  #image Size
  IMG_SIZE = 32

  #number of clusters 
  n = 2


  #cropped files to be taken for classification
  for k in filename_cropped:
    I = cv.imread(k)
    #to resize the files
    width = int(I.shape[1] * 75 / 100)
    height = int(I.shape[0] * 75 / 100)
    dim = (width, height)
    I=cv2.resize(I,(IMG_SIZE,IMG_SIZE))
    #storing the image in Numpy arr
    a = np.asarray(I,dtype=np.float32)
    #normalizing
    a = a/255.0
    reshaped_data = a.reshape(len(a),-1)
    reshaped_data=np.hstack(reshaped_data)
    
    arr = np.array(reshaped_data)
    arr.flatten()
    final.append(arr)
  #changing list to numpy arr
  final=np.array(final)

  # #team1_mask = [255, 0, 0] | [255, 255, 255] # replace with a mask for team1's jersey color
  # team2_mask = [0, 0, 0]  # replace with a mask for team2's jersey color
  # team1_colors = []
  # team2_colors = []
  # for image in final:
  #   team1_pixels = [color for color in image if np.array_equal(color, team1_mask)]
  #   team2_pixels = [color for color in image if np.array_equal(color, team2_mask)]
  #   team1_color = np.mean(team1_pixels, axis=0)
  #   team2_color = np.mean(team2_pixels, axis=0)
  #   team1_colors.append(team1_color)
  #   team2_colors.append(team2_color)
  #df_PCA=pd.DataFrame(final)
  #print(df_PCA.shape)
  #n_pca = 2  # set the desired number of PCA components
  #pca = PCA(n_components=n_pca)
  #final = pca.fit_transform(final)

  k_means = KMeans(n_clusters=n)
  #training a model
 
  # colors = KMeans.cluster_centers_.astype(int)
  # print(colors)
  if int(i.stem) <=150:
    final = np.vstack(final)
    model_Team = k_means.fit(final)
    print(final.shape)
  #colors = k_means.cluster_centers_.astype(int)
  
  #predicting the images
  Final_Lables=model_Team.predict(final)

  #naming the lables
  lables = ['Team1','Team2','Others']
  lables_dic= {0 :'Team1', 1:"Team2",2: "Others"}
  Final_Lables=[lables_dic.get(x, x) for x in Final_Lables]
  print(Final_Lables)


  
  shutil.copy("/content/Images/"+i.stem+".jpg","/content/Cropped_Images/")

  im=PIL.Image.open("/content/Cropped_Images/"+i.stem+".jpg")

  for m in range(0,count_row):
    h=df["ymax"].values[m]-df["ymin"].values[m]
    w=df["xmax"].values[m]-df["xmin"].values[m]
    y=df["ymin"].values[m]
    x=df["xmin"].values[m]
    draw = ImageDraw.Draw(im)
    if(Final_Lables[m]=='Team1'):
      draw.rectangle((x, y, x+w, y+h), outline=(255, 0, 0),width=2)
    elif(Final_Lables[m]=='Team2'):
      draw.rectangle((x, y, x+w, y+h), outline=(0, 255, 0),width=2)
    elif(Final_Lables[m]=='Others'):
      draw.rectangle((x, y, x+w, y+h), outline=(0, 0, 0),width=2)
    draw2 = ImageDraw.Draw(im)
    draw2.text((x, y),Final_Lables[m])
    im.save("/content/Cropped_Images/"+i.stem+".jpg")
  display(Image("/content/Cropped_Images/"+i.stem+".jpg"))
  #print(i.stem)

In [ ]:
import cv2
import os,glob
from pathlib import Path

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # FourCC code
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))
a = natsorted(glob.glob('/content/Images/*.jpg'))
# Load the images
images = []
for i in a:images.append(Path(i))
# for i in images:
#     images.append(cv2.imread(i))

# Write the images to the video
# for image in images:
#   out.write(image)

# # Release the VideoWriter
# out.release()
im = cv2.imread('/content/Images/0.jpg')
height=im.shape[0]
width=im.shape[1]
depth=im.shape[2]
print(width,height)

In [ ]:
import cv2
import os

# Create a VideoWriter object
output_video = cv2.VideoWriter('/content/output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30,(width,height ))

# Iterate through the images in the directory
for filename in natsorted(os.listdir('/content/Cropped_Images/')):
    if filename.endswith('.jpg'):
        # Read the image
        img = cv2.imread(os.path.join('/content/Cropped_Images', filename))
        # Resize the image if necessary
        img = cv2.resize(img, (width, height))
        # Write the image to the video
        output_video.write(img)

# Release the VideoWriter object
output_video.release()

In [ ]:
!rm -r '/content/Cropped_Images'

In [ ]:
filename_predictions